In [3]:
from pinecone import Pinecone, ServerlessSpec
import os, getpass
import tiktoken

In [4]:
api_key = "dcd9fb64-a64e-4195-b63c-478fbae6635a"
os.environ["PINECONE_API_KEY"] = api_key

In [15]:
os.environ["OPENAI_API_KEY"] = "sk-proj-0XT3GGvdjgAQUVMOQrhBT3BlbkFJEvyFh5Mjlm8NbuNB507I"
#sk-proj-Y9DaHbKZTzsLLzhhlIp9T3BlbkFJNKa2u8LqkmwcKS5JB4mO

In [7]:
pinecone_client = Pinecone()

# Crea un índice usando el cliente de Pinecone
pinecone_client.create_index(
    name="quickstarts",
    dimension=8, # Reemplaza con las dimensiones de tu modelo
    metric="euclidean", # Reemplaza con la métrica de tu modelo
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

## Creando un index (tabla) en Pinecone

In [9]:
from pinecone import Pinecone, ServerlessSpec
import os

# Configura la clave API de Pinecone en la variable de entorno
api_key = "dcd9fb64-a64e-4195-b63c-478fbae6635a"
os.environ["PINECONE_API_KEY"] = api_key

# Nombre del índice que deseas crear
index_name = "knowledge-base-stable-diffusions"

# Dimension del modelo, ajustado para Stable Diffusion
dimension = 1536

# Especificaciones del servidor donde se alojará el índice
spec = ServerlessSpec(
    cloud="aws",  # Ajusta según la nube que estés utilizando, "aws", "gcp", etc.
    region="us-east-1"  # Asegúrate de que la región esté disponible en tu plan
)

# Inicializa el cliente de Pinecone
pinecone_client = Pinecone()

# Crea un índice en Pinecone
pinecone_client.create_index(
    name=index_name, 
    dimension=dimension,
    spec=spec  # Incluye las especificaciones del servidor
)

print("Index " + index_name + " creado con éxito en Pinecone")


Index knowledge-base-stable-diffusions creado con éxito en Pinecone


## Estadísticas de un index

In [10]:
index = pinecone_client.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


## Lista de index en Pinecone

In [11]:
pinecone_client.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'knowledge-base-stable-diffusion-344da37.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'knowledge-base-stable-diffusion',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'host': 'knowledge-base-s3-344da37.svc.aped-4627-b74a.pinecone.io',
              'metric': 'euclidean',
              'name': 'knowledge-base-s3',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'host': 'knowledge-base-stable-diffusions-344da37.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'knowledge-base-stable-diffusions',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
      

## Cargando datos (pdf) y generando sus fragmentos

In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("pdf/ebook-stable-difussion-guia.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 400, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)

In [13]:
len(docs),docs

(23,
 [Document(page_content='ChatGPT + Stable DiffusionCómo crear Contenido con \nInteligencia Artificial', metadata={'source': 'pdf/ebook-stable-difussion-guia.pdf', 'page': 0}),
  Document(page_content='Índice\nIntroducción\nCómo puede ayudarte la Inteligencia Artiﬁcial a crear contenido\nQué es Stable Diffusion\nQué es ChatGPT03\n03\n04\n05\nStable Diffusion 06\nGuía ChatGPT\nCómo puedes usar ChatGPT\nEjemplos prácticos de ChatGPT\nAlternativas a ChatGPT19\n20\n22\n23Cómo puedes utilizar Stable Diffusion\nIntroducción a Dream Studio\nIntroducción a la versión NKMD de escritorio\nIntroducción a la versión Automatic 1111\nCómo crear imágenes con Stable Diffusion\nCómo escribir buenos prompts\nOtras herramientas basadas en Stable Diffusion07\n09\n10\n11\n15\n16\n18\nSobre el autor:\nSoy Xavier Mitjana, especialista en creación de contenido para \ncampañas de comunicación y marketing. También soy un apasionado de la divulgación y la innovación tecnológica. En mi canal de YouTube puedes

## Cargando embeddings a Pinecone

In [16]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)

Nos vamos a nuestra cuenta de Pinecone, para validar que se hayan insertados los embeddings

In [17]:
index = pinecone_client.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


## Consultando datos del index

![Imagen](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png)

## Muestra los 5 embeddings más relevantes con la pregunta

In [18]:
query = "¿qué es stable diffussion?"
docs = docsearch.similarity_search(query, k = 5) 
docs

[Document(page_content='02\nQué es \nStable Diffusion\nStable Diffusion es un modelo de aprendizaje automático \ndesarrollado por Stability AI para generar imágenes digitales \nde alta calidad a partir de descripciones en lenguaje natural. \nEl modelo se puede usar para diferentes tareas, como la \ngeneración de traducciones de imagen a imagen guiadas por \nmensajes de texto y la mejora de imágenes.\nA diferencia de modelos de la competencia como DALL-E, \nStable Diffusion es de código abierto1  y no limita \nartiﬁcialmente las imágenes que produce.2\nY este es el principal motivo por el que está adquiriendo gran \npopularidad. Ya que permite crear imágenes sin límites y crear \nincluso nuestros propios modelos para crear imágenes que se \najusten mejor a nuestros casos de uso.\nPor ejemplo imágenes con nuestra cara.\nY todo de un modo tan sencillo como introducir una indicación \nen texto plano y esperar unos segundos a que la inteligencia \nartiﬁcial la interprete en forma de imagen.

In [19]:
query = "¿en qué me puede ayudar la inteligencia artificial?"
docs = docsearch.similarity_search(query, k = 5)
docs

[Document(page_content='01\nCómo puede \nayudarte la Inteligencia Artiﬁcial \na crear contenido\nLa inteligencia artiﬁcial puede ayudarte a crear contenido \nproporcionando texto e imágenes a medida de tus \nindicaciones, ahorrándote tiempo y esfuerzo para que puedas \ncentrarte en la producción de contenido de calidad. Por \nejemplo, la inteligencia artiﬁcial puede proporcionar textos \nalrededor de una palabra clave, puede plantear la mejor \nmanera de estructurar un texos para mantener a los lectores \ninteresados, e incluso puede crear piezas de contenido por sí \nmisma.\nEn esta guía te explicaré como crear contenido a partir de 2 \nherramientas: Stable Diffusion y ChatGPT (GPT3).\nStable Diffusion es un modelo Open Source de generación de \nimágenes con Inteligencia Artiﬁcial.\nPor otro lado ChatGPT, es una herramienta basada en GPT 3.5, \nque permite crear textos a partir de establecer un diálogo con \nla Inteligencia Artiﬁcial.\nGuia Inteligencia Artiﬁcial - 03', metadata={'pag

In [20]:
print(docs[0].page_content)

01
Cómo puede 
ayudarte la Inteligencia Artiﬁcial 
a crear contenido
La inteligencia artiﬁcial puede ayudarte a crear contenido 
proporcionando texto e imágenes a medida de tus 
indicaciones, ahorrándote tiempo y esfuerzo para que puedas 
centrarte en la producción de contenido de calidad. Por 
ejemplo, la inteligencia artiﬁcial puede proporcionar textos 
alrededor de una palabra clave, puede plantear la mejor 
manera de estructurar un texos para mantener a los lectores 
interesados, e incluso puede crear piezas de contenido por sí 
misma.
En esta guía te explicaré como crear contenido a partir de 2 
herramientas: Stable Diffusion y ChatGPT (GPT3).
Stable Diffusion es un modelo Open Source de generación de 
imágenes con Inteligencia Artiﬁcial.
Por otro lado ChatGPT, es una herramienta basada en GPT 3.5, 
que permite crear textos a partir de establecer un diálogo con 
la Inteligencia Artiﬁcial.
Guia Inteligencia Artiﬁcial - 03


In [21]:
docs = docsearch.similarity_search_with_score(query, k = 3)
docs

[(Document(page_content='01\nCómo puede \nayudarte la Inteligencia Artiﬁcial \na crear contenido\nLa inteligencia artiﬁcial puede ayudarte a crear contenido \nproporcionando texto e imágenes a medida de tus \nindicaciones, ahorrándote tiempo y esfuerzo para que puedas \ncentrarte en la producción de contenido de calidad. Por \nejemplo, la inteligencia artiﬁcial puede proporcionar textos \nalrededor de una palabra clave, puede plantear la mejor \nmanera de estructurar un texos para mantener a los lectores \ninteresados, e incluso puede crear piezas de contenido por sí \nmisma.\nEn esta guía te explicaré como crear contenido a partir de 2 \nherramientas: Stable Diffusion y ChatGPT (GPT3).\nStable Diffusion es un modelo Open Source de generación de \nimágenes con Inteligencia Artiﬁcial.\nPor otro lado ChatGPT, es una herramienta basada en GPT 3.5, \nque permite crear textos a partir de establecer un diálogo con \nla Inteligencia Artiﬁcial.\nGuia Inteligencia Artiﬁcial - 03', metadata={'pa

## Eliminando index

In [22]:
pinecone_client.delete_index(index_name)